In [5]:
import pandas as pd
from sqlalchemy import create_engine, text
import sqlite3

# Connect our bd
engine = create_engine('sqlite:///../data/company.db')
#connection = sqlite3.connect('../data/company.db')

### Connect to data base

In [6]:
#def run_query(query):
#   with engine.connect() as connection:
#      result = pd.read_sql_query(query, connection)
# return result

# Función para ejecutar consultas en la base de datos
def run_query(query: str):
    with engine.begin() as conn:
        # Verificar si la consulta es de modificación
        if query.strip().lower().startswith(('insert', 'update', 'delete')):
            conn.execute(text(query))  # Ejecuta la consulta
            return "Query executed successfully."
        else:
            # Para consultas de solo lectura
            result = pd.read_sql_query(query, conn)
            return result


# Advances Sub queries 

### Simple queries

In [7]:
#! pending
table = "employee"
query1 = f"""
    SELECT * FROM {table}
"""

result1 = run_query(query1)
print("simple query")
result1

simple query


,id,name,last_name,document,department,manager,salary,experience
0,100,David,Ruiz,8325476,CEO,0,5000000,SENIOR
1,101,Ana,García,10234567,Direccion comercial,100,3800000,SEMISENIOR
2,102,Carlos,López,10345678,Direccion Financiera,100,3000000,SEMISENIOR
3,103,María,Rodríguez,10456789,Direccion Marketing,100,4200000,SENIOR
4,104,José,Martínez,10567890,Cartera,102,2600000,JUNIOR
5,105,Laura,Hernández,10678901,Analista,102,3800000,SEMISENIOR
6,106,Pedro,Gómez,10789012,Publicidad,103,3200000,SEMISENIOR
7,107,Marta,Sánchez,10890123,Tesoreria,102,4400000,SENIOR
8,108,Luis,Díaz,10901234,Digital,103,1800000,JUNIOR
9,109,Carmen,Ramírez,11012345,Contador,102,2300000,JUNIOR


### Sub query 1
Sort rows based on an order criterion

In [8]:
row = "salary"

select_employees = f"""
    SELECT id, 
    last_name, 
    name, 
    salary, 
    RANK() OVER (ORDER BY {row} DESC) AS ranking
    FROM employee
"""
query2 = f"""
    {select_employees}
    ORDER BY ranking
"""

result2 = run_query(query2)
print("Sub query 2")
result2

Sub query 2


,id,last_name,name,salary,ranking
0,100,Ruiz,David,5000000,1
1,107,Sánchez,Marta,4400000,2
2,103,Rodríguez,María,4200000,3
3,101,García,Ana,3800000,4
4,105,Hernández,Laura,3800000,4
5,106,Gómez,Pedro,3200000,6
6,102,López,Carlos,3000000,7
7,110,Pérez,Juan,3000000,7
8,104,Martínez,José,2600000,9
9,111,Ruiz,David,2400000,10


### 3. Example 3: Show the 5 first rows of a data set


CTE: define temporary queries based on what already exists.


In [9]:
condition = "ranking <= 5"
second_select = f"""
    SELECT 
        id, 
        last_name, 
        name, 
        salary
        FROM ranking_employees     
"""
query3 = f"""
    WITH ranking_employees AS (
        {select_employees}
    )
    {second_select}
    WHERE {condition} 
    ORDER BY ranking
"""
result3 = run_query(query3)
print("Example 3")
result3

Example 3


,id,last_name,name,salary
0,100,Ruiz,David,5000000
1,107,Sánchez,Marta,4400000
2,103,Rodríguez,María,4200000
3,101,García,Ana,3800000
4,105,Hernández,Laura,3800000


### 4. Example 4: List the las 5 rows of a data set

In [10]:
query4 = f"""
    WITH ranking_employees AS (
        {select_employees}
    )
    {second_select}
        WHERE ranking <= 5
        ORDER BY ranking DESC """
result4 = run_query(query4)
print("Example 4")
result4

Example 4


,id,last_name,name,salary
0,101,García,Ana,3800000
1,105,Hernández,Laura,3800000
2,103,Rodríguez,María,4200000
3,107,Sánchez,Marta,4400000
4,100,Ruiz,David,5000000


### 5. Example 5: List the second highest row of queried data 
Another way: ROW_NUMBER() OVER (ORDER BY salary DESC) AS ranking

In [11]:
condition = "ranking == 2"
query5 = f"""
    WITH ranking_employees AS (
        {select_employees}
    )
    {second_select}
    WHERE {condition}
    ORDER BY ranking DESC """
result5 = run_query(query5)
print("Example 5")
result5

Example 5


,id,last_name,name,salary
0,107,Sánchez,Marta,4400000


### Median
<div class="alert alert-danger">Missing</div>

### 6. Example 6...
Review the median operation

In [12]:
select_employees_ntile = """
    SELECT 
        id,
        last_name, 
        name, 
        salary, 
        NTILE(2) OVER (ORDER BY salary ASC) AS ntile
    FROM employee
"""

In [13]:
query6 = f"""
    WITH ranking_employees AS (
        {select_employees_ntile}
    )
    SELECT 
        id, 
        last_name, 
        name, 
        salary
        FROM ranking_employees
        WHERE ntile = 1
        ORDER BY salary
        """
result6 = run_query(query6)
print("Example 6")
result6

Example 6


,id,last_name,name,salary
0,108,Díaz,Luis,1800000
1,109,Ramírez,Carmen,2300000
2,111,Ruiz,David,2400000
3,113,Ruiz,David,2400000
4,115,Ruiz,David,2400000
5,104,Martínez,José,2600000
6,102,López,Carlos,3000000


### missing the text

In [14]:
select_employees_ntile = """
    SELECT 
        id,
        last_name, 
        name, 
        salary, 
        NTILE(10) OVER (ORDER BY salary ASC) AS ntile
    FROM employee
"""

second_select = """
    SELECT 
        id, 
        last_name, 
        name, 
        salary
        FROM ranking_employees
        WHERE ntile = 5
        ORDER BY salary
"""

### 7. Example 7: Show the median of a consulted data set

In [15]:
query7 = f"""
    WITH ranking_employees AS (
        {select_employees_ntile}
    )
    {second_select}
    """
result7 = run_query(query7)
print("Example 7")
result7

Example 7


,id,last_name,name,salary
0,106,Gómez,Pedro,3200000


### 8. Example 8: To list between 40% and 70% of data set

In [16]:
second_select = """
    SELECT 
        id, 
        last_name, 
        name, 
        salary
        FROM ranking_employees
        WHERE ntile > 4 AND ntile <= 7
        ORDER BY salary
"""

In [17]:
query8 = f"""
    WITH ranking_employees AS (
        {select_employees_ntile}
    )
    {second_select}    
    """
result8 = run_query(query8)
print("Example 8")
result8

Example 8


,id,last_name,name,salary
0,106,Gómez,Pedro,3200000
1,101,García,Ana,3800000
2,105,Hernández,Laura,3800000



# october 28th - 2024

### 9. Example 9: Show all register up the ...

In [18]:
select_employee = """
    SELECT last_name,
    name,
    salary
    FROM employee
"""

In [19]:
query9 = f"""
    {select_employee}
    WHERE salary > (SELECT AVG(salary) FROM employee)
"""
result9  = run_query(query9)

# result AVG
query9_2 = "SELECT AVG(salary) FROM employee"
result9_2 = run_query(query9_2)

print("Result AVG: ",result9_2)
print("Select Example 9 with AVG")
result9

Result AVG:      AVG(salary)
0  3.164286e+06
Select Example 9 with AVG


,last_name,name,salary
0,Ruiz,David,5000000
1,García,Ana,3800000
2,Rodríguez,María,4200000
3,Hernández,Laura,3800000
4,Gómez,Pedro,3200000
5,Sánchez,Marta,4400000


### 10. Example 10....

In [20]:
query10 = f"""
    {select_employee}
    WHERE salary < (SELECT AVG(salary) FROM employee)
"""
result10  = run_query(query10)

# result AVG
print("Result AVG: ",result9_2)
print("Select Example 10 with AVG")
result10

Result AVG:      AVG(salary)
0  3.164286e+06
Select Example 10 with AVG


,last_name,name,salary
0,López,Carlos,3000000
1,Martínez,José,2600000
2,Díaz,Luis,1800000
3,Ramírez,Carmen,2300000
4,Pérez,Juan,3000000
5,Ruiz,David,2400000
6,Ruiz,David,2400000
7,Ruiz,David,2400000


### 11. Example 11: ....

In [21]:
id_manager = 102
query11 = f"""
    SELECT 
        name,
        last_name,
        department
        FROM employee
        WHERE id IN (
            SELECT id
            FROM employee
            WHERE manager = {id_manager}
        )
"""
# boss the last employees
query11_1 = f"""
    SELECT 
        name,
        last_name,
        department
        FROM employee
        WHERE id = {id_manager}
"""
result11_1 = run_query(query11_1)

result11 = run_query(query11)

print("Example 12: The department boss")
print(result11_1)
print("-----------------------------------")
print("Example 11: The employees are:")
result11

Example 12: The department boss
     name last_name            department
0  Carlos     López  Direccion Financiera
-----------------------------------
Example 11: The employees are:


,name,last_name,department
0,José,Martínez,Cartera
1,Laura,Hernández,Analista
2,Marta,Sánchez,Tesoreria
3,Carmen,Ramírez,Contador
4,David,Ruiz,Analista
5,David,Ruiz,Analista
6,David,Ruiz,Analista


### 11.2 Example 11(Version 2)

In [22]:
# Define the manager id
id_manager = 102

# Combine the two queries into one
query11_2 = f"""
    SELECT
        CASE
            WHEN id = {id_manager} THEN 'Department Boss'
            ELSE 'Employees'
        END AS Title,
        name,
        last_name,
        department
    FROM employee
    WHERE id = {id_manager}
    OR id IN (
        SELECT id
        FROM employee
        WHERE manager = {id_manager}
    )
"""

# Execute the query11_2
result = run_query(query11_2)

# Split the results into two DataFrames
boss_df = result[result['Title'] == 'Department Boss'].drop(columns=['Title'])
employees_df = result[result['Title'] == 'Employees'].drop(columns=['Title'])

# Print the results
print("Example 12: The department boss")
print(boss_df.to_string(index=False))
print("-----------------------------------")
print("Example 11: The employees are:")
print(employees_df.to_string(index=False))

Example 12: The department boss
  name last_name           department
Carlos     López Direccion Financiera
-----------------------------------
Example 11: The employees are:
  name last_name department
  José  Martínez    Cartera
 Laura Hernández   Analista
 Marta   Sánchez  Tesoreria
Carmen   Ramírez   Contador
 David      Ruiz   Analista
 David      Ruiz   Analista
 David      Ruiz   Analista


### 12. Example 12: ....

In [23]:
query12 = """
    SELECT 
        e1.name || " " || e1.last_name AS manager_name, 
        e2.name  || " " || e2.last_name AS employee_name
        FROM employee e1
        JOIN employee e2 
        ON e1.id == e2.manager
        WHERE e1.id = 102
"""
result12 = run_query(query12)
print("Example 12")
result12

Example 12


,manager_name,employee_name
0,Carlos López,José Martínez
1,Carlos López,Laura Hernández
2,Carlos López,Marta Sánchez
3,Carlos López,Carmen Ramírez
4,Carlos López,David Ruiz
5,Carlos López,David Ruiz
6,Carlos López,David Ruiz


# october 30th - 2024

### 13. Example 13....

In [24]:
#
    #? Insert into employee
insert_into_employee = """
    INSERT INTO employee (
    id, 
    name, 
    last_name, 
    document, 
    department, 
    manager, 
    salary,
    experience) 
"""

In [25]:
query13_2 = f"""
    {insert_into_employee}
    VALUES 
    (113, 'David', 'Ruiz', 213000023, 'Analista', 102, 2400000,'SENIOR');
"""
run_query(query13_2)


IntegrityError: (sqlite3.IntegrityError) UNIQUE constraint failed: employee.id
[SQL: 
    
    INSERT INTO employee (
    id, 
    name, 
    last_name, 
    document, 
    department, 
    manager, 
    salary,
    experience) 

    VALUES 
    (113, 'David', 'Ruiz', 213000023, 'Analista', 102, 2400000,'SENIOR');
]
(Background on this error at: https://sqlalche.me/e/20/gkpj)

In [3]:
# update a register, this was save with an error
query_update = """
    UPDATE employee
    SET document = '11111111'
    WHERE id = 111
"""
result_update = run_query(query_update)

In [4]:
query13 = """
    SELECT department, experience,
    SUM(salary) AS total_salary
    FROM employee
    GROUP BY department, experience
"""

result13 = run_query(query13)
print("Example 13")
result13

Example 13


,department,experience,total_salary
0,Analista,SEMISENIOR,3800000
1,CEO,SENIOR,5000000
2,Cartera,JUNIOR,2600000
3,Contador,JUNIOR,2300000
4,Digital,JUNIOR,1800000
5,Direccion Financiera,SEMISENIOR,3000000
6,Direccion Marketing,SENIOR,4200000
7,Direccion comercial,SEMISENIOR,3800000
8,Publicidad,SEMISENIOR,3200000
9,Talento Humano,SEMISENIOR,3000000


### 14. Example 14: Create a conditional summation and nested conditions

In [29]:
query14 = """
    SELECT 
        SUM(CASE 
            WHEN department IN ('Analista', 'Talento Humano')
            THEN salary 
            ELSE 0
            END) AS total_salary_analita_and_human_talent,
        SUM(CASE 
            WHEN department IN ('Tesoreria', 'Cartera')
            THEN salary 
            ELSE 0
            END) AS total_salary_tesoreria_and_cartera
        FROM employee
"""
result14 = run_query(query14)
print("Example 14\n")
#print("Total salaries of Analyst and Human Talent personnel\n")
#Forma 1
salary1 = result14.iloc[0,0]
print(f"\nTotal salaries of Analyst and Human Talent is {salary1}\n")

#Forma 2
print("Total salaries of Analyst and Human Talent:", f'$ {result14.iloc[0,0]:,.0f}',"\n")

#Forma 3
result14

Example 14


Total salaries of Analyst and Human Talent is 14000000

Total salaries of Analyst and Human Talent: $ 14,000,000 



,total_salary_analita_and_human_talent,total_salary_tesoreria_and_cartera
0,14000000,7000000


### 16. Example 16: Group rows according to a salary range

In [4]:
query16 = """
    SELECT
        CASE
            WHEN salary <= 2500000 THEN 'Level 1'
            WHEN salary > 2500000 AND salary <= 3800000 THEN 'Level 2'
            WHEN salary > 3800000 THEN 'Level 3'
        END AS level_salary,
        COUNT(*) AS quantity_employees
        FROM employee
        GROUP BY
        CASE
            WHEN salary <= 2000000 THEN 'Level 1'
            WHEN salary > 2500000 AND salary <= 3800000 THEN 'Level 2'
            WHEN salary > 3800000 THEN 'Level 3'
        END
"""
result16 = run_query(query16)
print("Example 16")
result16


NameError: name 'run_query' is not defined